In [1]:
import pandas as pd
import numpy as np
import re

pd.options.display.max_columns = None

In [2]:
df = pd.read_csv("../data/01-tiburon_4.csv", index_col = 0)
df.head(2)

,year,type,country,activity,age,species,month,fatal,gender
0,2018,Boating,usa,Paddling,57,White shark,Jun,N,F
1,2018,Unprovoked,brazil,Swimming,18,Tiger shark,Jun,Y,M


# Pair Programming Limpieza II
​
Antes de seguir, recordamos las preguntas que nos planteamos al principio del _pair programming_** de EDA para dirigir nuestro análisis.**

- ¿Es Australia es el sitio más peligroso y letal para estar relajada en la playa?
- ¿Cuál es el rango de edad que sufre la mayoría de los ataques?
- Independientemente de la edad, sufren los hombres más ataques que las mujeres?
- ¿En qué mes ocurren más ataques?
- ¿Cuál es la relación entre la especie y el tipo de ataque (si es fatal o no)?
- ¿Cómo han evolucionado los ataques a lo largo del tiempo?

De todo esto, nos damos cuenta que solo tenemos que limpiar algunas de las columnas, en concreto age, species, country, fatal, year, sex. Si reducimos esto a una tabla para saber que tenemos ya limpito y que no. 

**Actualizamos esta tabla ya que en el ejercicio de pair de Limpieza I ya dejamos algunas columnas limpitas:

Por lo tanto, en este ejercicio nos queda por limpiar las columnas de age y species, así que manos a la obra. Vamos a ver que podemos hacer para cada una de las columnas y os dejaremos algunas pistas 😉.

1. Columna de species: si exploramos esta columna en detalle nos podemos dar cuenta que tenemos muchos valores únicos y esto hace que sea muy difícil trabajar con esta columna. Lo que vamos a hacer es clasificar los tiburones en 5 especies diferentes, las más comunes, que incluyen el tiburón blanco (White), el tiburón tigre (Tiger ), el tiburón gris (Grey), el tiburón limón (Lemon) y el tiburón toro (Bull). 

El resto de las especies las incluiremos en un único grupo que podremos llamar "Unspecified". Ahora nos podemos sentir un poco abrumadas y no saber como enfrentarnos a este reto, pero don't worry, os dejamos por aquí unas 

💡 pistas 💡 para que os ayuden a entender cómo hacerlo.

- Los valores de las columnas son strings por lo que podremos usar regex para buscar palabras clave en cada celda y asignarlo a una de las categorías que hemos definido previamente.

imaginemos que el valor de una celda es el siguiente

'White shark, 3.5 m'

tendremos que buscar el patrón de regex que nos permita extraer White shark de ese string y que nos devuelva solo White shark. 

 patrón que podriamos usar es: 

patron_blanco = r".*[Ww](hite|HITE).*" # esto podría ser así porque puede estar en mayúsculas o en minúsculas. 

de la misma forma que hemos sacado el patron para el tiburón blanco, tendremos que sacar los patrones para las otras 4 especies que queremos "encontrar". 

- Tendremos que crearnos una función que aplicaremos sobre nuestra columna species_ para que nos devuelva una nueva columna con los valores clasificados en función de los patrones de regex que hayamos definido.

In [3]:
df.head()

,year,type,country,activity,age,species,month,fatal,gender
0,2018,Boating,usa,Paddling,57,White shark,Jun,N,F
1,2018,Unprovoked,brazil,Swimming,18,Tiger shark,Jun,Y,M
2,2018,Unprovoked,usa,Walking,15,"Bull shark, 6'",May,N,M
3,2018,Provoked,australia,Feeding sharks,32,Grey reef shark,May,N,M
4,2018,Invalid,england,Fishing,21,Invalid incident,May,N,M


In [4]:
# Creamos una función que nos busque un patrón por cada uno de los 5 principales tipos de especie, y convierta el nombre de la especie en algo más legible
def esp_tiburon(x):
    # Mediante un dicionario, pasamos por cada key(Nombre final), un value(Patrón)
    dict_especies= {"White": ".*[Ww](hite|HITE).*", "Tiger":".*[Tt](iger|IGER).*", "Grey": ".*[Gg](rey|REY).*", "Bull": ".*[bB](ull|ULL).*", "Lemon": ".*[lL](emon|EMON).*", "Unspecified": ".*"}
    try:
        for k,v in dict_especies.items(): # iteramos por el diccionario y buscamos con re.findall en cada fila de la columna del DF
            if re.findall(v,x):             # Cuando encuentra el patrón (value), nos devuelve el nombre final (key)
                return k
    except:
        return "Unspecified"    # para los nan, también nos devolverá "Unspecified"

In [5]:
# Aplicamos la función a la columna que nos interesa, en este caso, species
df["species"] = df["species"].apply(esp_tiburon)

In [6]:
# Comprobamos los datos
df["species"].value_counts()

Unspecified    982
White          385
Tiger          157
Bull           107
Grey            30
Lemon            8
Name: species, dtype: int64

In [7]:
# Ya no tenemos valores null
df["species"].isnull().sum()

0

In [8]:
df.head(2)

,year,type,country,activity,age,species,month,fatal,gender
0,2018,Boating,usa,Paddling,57,White,Jun,N,F
1,2018,Unprovoked,brazil,Swimming,18,Tiger,Jun,Y,M


2. Columna de age: es una columna de tipo string pero debería ser de tipo integer. Además, en esta columna nos vamos a encontrar con algunos errores tipográficos, estos incluyen:
- Edad en formato string
- Edades separadas por &, or, to, >
- Edades con ?

Vamos con algunas 💡 pistas 💡 para que os ayuden a entender cómo hacerlo:

- Primero tendremos que eliminar todos esos símbolos especiales que nos aparecen. De nuevo, podremos usar regex para extraer unicamente los números que es lo que nos interesa. Usar este regex en una función para sacar solo los números.
- Puede que os salga un error similar a este:
TypeError: expected string or bytes-like object
Para solucionar este problema, antes de nada tendréis que ejecutar este código para que no os de error:
df['nombre_columna'] = df['nombre_columna'].astype(str)


- Una vez que hayáis extraido los números, os daréis cuenta que hay celdas que tienen más de una edad. Tendréis que decidir que hacer en esos casos. Os dejamos por aquí una posible opción usando un método de Pandas que os puede resultar super útil. El método explode, aqui.

- Por último cambiad el tipo de la columna de string a integer.

In [9]:
# Comprobamos de nuevo el tipo de datos que tenemos en nuestro DF
df.dtypes

year         int64
type        object
country     object
activity    object
age         object
species     object
month       object
fatal       object
gender      object
dtype: object

In [10]:
# Convertimos la columna "age" en tipo string
df['age'] = df['age'].astype(str)

In [11]:
# Definimos una función que nos busca un patrón de dígitos
def numeros(x):
    patron_digitos = "\d+"
    try:
        # De la lista que nos devuelve al pasar el patrón, nos quedamos solo con los que estén en pos 0, 
        # deshaciendonos así de la segunda edad en los casos de x or y, o x & y
        return re.findall(patron_digitos, x)[0]  
    except:
        return np.nan

In [12]:
# Aplicamos la función a la columna "age"
df["age"] = df["age"].apply(numeros)

In [13]:
# Convertimos la columna age a integer
df["age"] = df["age"].astype("Int64", errors= "raise")

In [14]:
# Podemos ver que está bien, ya que teníamos 152 null y solo ha subido en 4, que podemos suponer que eran los datos tipo "Teen" o 'young'
df.age.isnull().sum()

156

In [15]:
# Comprobamos el tipo de datos
df.age.dtypes

Int64Dtype()

In [16]:
df.head()

,year,type,country,activity,age,species,month,fatal,gender
0,2018,Boating,usa,Paddling,57,White,Jun,N,F
1,2018,Unprovoked,brazil,Swimming,18,Tiger,Jun,Y,M
2,2018,Unprovoked,usa,Walking,15,Bull,May,N,M
3,2018,Provoked,australia,Feeding sharks,32,Grey,May,N,M
4,2018,Invalid,england,Fishing,21,Unspecified,May,N,M


3. Guarda el csv con las columnas limpias para seguir trabajando con este dataframe limpio.

In [17]:
df.to_csv("../data/02-tiburon_4.csv")